## Problems:
## 1)Loss function : Dice + cross entropy
## 2)We dont need accurate segmentation instead we could just work with a bounding box metric
## 3)For the model we used vgg backbone with decoder
## 4)We add a distance based weighted loss term
## 5)Made new mask which have a larger area now
## 6)Also its important to consider the breast mask and make sure our region of focus is inside the breast.

In [25]:
#Pytorch
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

#Torchvision
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader

#Image Processing
import matplotlib.pyplot as plt
from skimage import io, transform, color
import PIL
from PIL import Image
import augmentations
from augmentations import *

#Others
import sklearn.metrics
from sklearn.metrics import *
import numpy as np
import pandas as pd
import cv2
import time
import os
import copy
from model_summary import *
import pretrainedmodels
import tqdm
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings("ignore")

## Dataloader

class dataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):

        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.mask_dir = '../Data/mask_orient_cropped/'
        self.dist_mask_dir = '../Data/distance_mask_orient_cropped/'
        
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):

        #get the image
        img_name = os.path.join(self.root_dir,self.data_frame.iloc[idx]['name'])
        image = Image.open(img_name)
        
        #get the mask
        mask_name = os.path.join(self.mask_dir,self.data_frame.iloc[idx]['name'].replace('.j','_mask.j'))
        mask = io.imread(mask_name)
        
        #get the distance mask
        dist_mask_name = os.path.join(self.dist_mask_dir,self.data_frame.iloc[idx]['name'].replace('.j','_mask.j'))
        dist_mask = io.imread(dist_mask_name)

        #add the distance mask as the final channel of the 
        mask = np.array([mask,mask,dist_mask]).transpose((1,2,0))
        mask = Image.fromarray(mask)     

        if self.transform:
            image,mask = self.transform(image,mask)
                
        mask_final = mask[0,:,:]
        mask_final[mask_final<0.5] = 0
        mask_final[mask_final>0.5] = 1
        
        dist_mask = mask[2,:,:]
        
        return {'image':image, 'mask':mask_final, 'dist_mask':dist_mask, 'name':self.data_frame.iloc[idx]['name']}
    

def get_dataloader(data_dir, train_csv_path, img_mean, img_std, batch_size=1):

    data_transforms = {
        'train': Compose([
            RandomHorizontallyFlip(0.5),
            RandomVerticallyFlip(0.5),
            RandomTranslate((0.1,0.1)),
            RandomRotate(15),
            ToTensor(),
            Normalize(img_mean,img_std)
        ]),
        'valid': Compose([
            ToTensor(),
            Normalize(img_mean,img_std)
        ]),
        'test': Compose([
            ToTensor(),
            Normalize(img_mean,img_std)        
        ])
    }

    image_datasets = {}
    dataloaders = {}
    dataset_sizes = {}

    for x in ['train', 'valid', 'test']:
        if x == 'train':
            bs = batch_size
            sh = True
        elif x == 'valid':
            bs = batch_size
            sh = False
        else:
            bs = 1
            sh = False
        image_datasets[x] = dataset(train_csv_path.replace('train',x),root_dir=data_dir,transform=data_transforms[x])
        dataloaders[x] = torch.utils.data.DataLoader(image_datasets[x], batch_size=bs,shuffle=sh, num_workers=8)    
        dataset_sizes[x] = len(image_datasets[x])

    device = torch.device("cuda:0")

    return dataloaders,dataset_sizes,image_datasets,device

# a,_,_,_ = get_dataloader('../Data/CBIS-DDSM_classification_orient_cropped/','../CSV/gain_train.csv',[0,0,0],[1,1,1],1)

class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

class UnetBlock(nn.Module):
    def __init__(self, up_in, x_in, n_out):
        super().__init__()
        up_out = x_out = n_out//2
        self.x_conv  = nn.Conv2d(x_in,  x_out,  1)
        self.tr_conv = nn.ConvTranspose2d(up_in, up_out, 2, stride=2)
        self.bn = nn.BatchNorm2d(n_out)
        
    def forward(self, up_p, x_p):
        up_p = self.tr_conv(up_p)
        x_p = self.x_conv(x_p)
        cat_p = torch.cat([up_p,x_p], dim=1)
        return self.bn(F.relu(cat_p))

class Unet34(nn.Module):
    def __init__(self, rn):
        super().__init__()
        self.rn = rn
        self.sfs = [SaveFeatures(rn[0][i]) for i in [12,22,32,42]]
        self.up1 = UnetBlock(512,512,256)
        self.up2 = UnetBlock(256,512,256)
        self.up3 = UnetBlock(256,256,256)
        self.up4 = UnetBlock(256,128,256)
        self.up5 = nn.ConvTranspose2d(256, 1, 2, stride=2)
        
    def forward(self,x):
        x = F.relu(self.rn(x))
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x)
        return x
    
    def close(self):
        for sf in self.sfs: sf.remove()

def build_model():
    v = models.vgg16_bn(pretrained=True)
    v1 = nn.Sequential(*list(v.children())[:-1])
    m = Unet34(v1)
    return m

def get_IoU(pred, targs):
    return 2*(pred*targs).sum() / ((pred+targs).sum())# - (pred*targs).sum())

def new_metric(preds,targs,device,patch_size=64):
    
    mask = targs
    b,c,h,w = preds.size()
    
#     conv_filter = torch.ones((1, 1, 5, 5))/25
#     conv_filter = conv_filter.to(device)
#     filtered_preds = F.conv2d(preds, conv_filter, padding=2)
#     max_loc = torch.argmax(filtered_preds)
#     h_loc = int(max_loc%h)
#     w_loc = int(max_loc/h)
    temp = preds
    temp = temp.detach().cpu().numpy()
    temp = temp.squeeze()
    temp[temp>0] = 1
    temp[temp<0] = 0
    #import pdb;pdb.set_trace()
    hs,ws = np.where(temp==1)
    try:
        h_loc = int(hs.mean())
        w_loc = int(ws.mean())
    except:
        return 0
    
    h_up = max(0,h_loc-patch_size//2)
    h_bottom = min(h,h_loc+patch_size//2)
    
    w_left = max(0,w_loc-patch_size//2)
    w_right = min(w,w_loc+patch_size//2)
    
    if h_up == 0:
        h_bottom = min(patch_size,h)
    elif h_bottom == h:
        h_up = max(0,-patch_size+h)
        
    if w_left == 0:
        w_right = min(patch_size,w)
    elif w_right == w:
        w_left = max(0,-patch_size+w)
    
    mask_loc = mask[0,h_up:h_bottom,w_left:w_right]
    return 1.0*mask_loc.sum()/(mask.sum()+1)
    
    
class SoftDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(SoftDiceLoss, self).__init__()

    def forward(self, logits, targets):
        smooth = 1
        num = targets.size(0)
        probs = F.sigmoid(1*(logits))
        m1 = probs.view(num, -1)
        m2 = targets.view(num, -1)
        intersection = (m1 * m2)

        score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        score = 1 - score.sum() / num
        return score
    
class DistanceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DistanceLoss, self).__init__()

    def forward(self, logits, targets):
    
        num = targets.size(0)
        probs = F.sigmoid(logits)
        m1 = probs.view(num, -1)
        m2 = targets.view(num, -1)
        intersection = (m1 * m2)

        score = intersection.sum()/m2.sum()
        return score
    
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        
    def forward(self, input, target):
        input = input.view(-1)
        target = target.view(-1)
        target_a = target*self.alpha
        target_1_a = (1-target)*(1-self.alpha)
        
        pt = F.sigmoid(input)
    
        loss = target_a*((1-pt)**self.gamma)*torch.log(pt+1e-8) + target_1_a*((pt)**self.gamma)*torch.log(1-pt+1e-8) 
        loss = -loss.mean()
        
        return loss   
    
class VGG_unet_bc():
    def __init__(self):
        
        #Initialization
        self.data_dir = '../Data/CBIS-DDSM_classification_orient_cropped/'
        self.train_csv = '../CSV/mass_segmentation_train.csv'
        self.num_epochs = 20
        self.batch_size = 1
        self.img_mean = [0.253, 0.238, 0.234]
        self.img_std = [0.272, 0.268, 0.262]
        self.exp_name = 'Weights/VGG_unet_cbis_ddsm_distloss'
        
        #Define the three models
        self.model = build_model()
        
        #Put them on the GPU
        self.model = self.model.cuda()
        #self.model.load_state_dict(torch.load(self.exp_name+'.pt'))
        
        #Get the dataloaders
        self.dataloaders,self.dataset_sizes,self.dataset,self.device = get_dataloader(self.data_dir,self.train_csv,\
                                                        self.img_mean,self.img_std,self.batch_size)
        

        self.optimizer = optim.Adam(self.model.parameters(),lr=1e-4, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
        
        #self.lr_scheduler = CosineAnnealingLR(self.optimizer,T_max=10,eta_min=1e-4)
    
        
    def train(self):
        
        since = time.time()
        best_epoch_acc = 0.0
        best_epoch_iou = 0.0
        best_epoch_metric = 0.0
        
        for epoch in range(self.num_epochs):
            print('Epoch {}/{}'.format(epoch, self.num_epochs - 1),flush=True)
            print('-' * 10,flush=True)

            # Each epoch has a training and validation phase
            for phase in ['train', 'valid']:
                if phase == 'train':
                    
                    #Set the models to training mode
                    #self.lr_scheduler.step()
                    self.model.train()
                
                else:
                    #Set the models to evaluation mode
                    self.model.eval()
                    
                #Keep a track of all the three loss
                running_loss = 0.0
                running_dice_loss = 0.0
                running_dist_loss = 0.0
                
                #Metrics : predictor auc and selector iou
                running_iou = 0
                running_new_metric = 0
                
                #tqdm bar
                pbar = tqdm(total=self.dataset_sizes[phase])

                # Iterate over data.
                for sampled_batch in self.dataloaders[phase]:

                    inputs = sampled_batch['image']
                    mask = sampled_batch['mask']
                    dist_mask = sampled_batch['dist_mask']
                   
                    #Input needs to be float and labels long
                    inputs = inputs.float().to(self.device)
                    mask = mask.to(self.device)
                    dist_mask = dist_mask.to(self.device)
                    
                    # zero the parameter gradients
                    self.optimizer.zero_grad()
                
                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        
                        outputs = self.model(inputs)
                        
                        out4loss = outputs.view(-1)
                        mask4loss = mask.view(-1)
                        dm4loss = dist_mask.view(-1)
                        
                        z_c = float((mask==0).sum())
                        o_c = float((mask==1).sum())
                        alpha = z_c/(z_c+o_c)
                        #ce_loss = nn.BCEWithLogitsLoss()(out4loss,new_mask4loss)
                        ce_loss = FocalLoss(2,alpha)(outputs,mask)
                        dice_loss = SoftDiceLoss()(outputs,mask)
                        dist_loss = DistanceLoss()(outputs,dist_mask)
                        loss = ce_loss + 2*dice_loss + dist_loss
                        
                        #print((F.sigmoid(out4loss)*dm4loss).sum(),(outputs>0).sum(),maskdm4loss.sum())
                        #import pdb;pdb.set_trace()
                        
                        # backward + optimize only if in training phase
                        if phase == 'train':
                            
                            loss.backward()
                            self.optimizer.step()
                                    
                    out4iou = out4loss
                    out4iou[out4loss>0] = 1
                    out4iou[out4loss<0] = 0
                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_dice_loss += dice_loss.item() * inputs.size(0)
                    running_dist_loss += dist_loss.item() * inputs.size(0)
                    running_iou += get_IoU(out4loss,mask4loss) * inputs.size(0)
                    running_new_metric += new_metric(outputs,mask,self.device,64) * inputs.size(0)
                    #print(running_new_metric)
                    #print(running_iou)
                    
                    pbar.update(inputs.shape[0])
                pbar.close()

                epoch_loss = running_loss / self.dataset_sizes[phase]
                epoch_dice_loss = running_dice_loss / self.dataset_sizes[phase]
                epoch_dist_loss = running_dist_loss / self.dataset_sizes[phase]
                epoch_iou = running_iou / self.dataset_sizes[phase]
                epoch_new_metric = running_new_metric / self.dataset_sizes[phase]
                
                print('{} Loss: {:.4f} Dice_Loss: {:.4f} Dist_Loss: {:.4f} IoU: {:.4f} New Metric: {:.4f}'.format(
                    phase, epoch_loss, epoch_dice_loss, epoch_dist_loss, epoch_iou, epoch_new_metric))

                # deep copy the model
                if phase == 'valid' and epoch_new_metric > best_epoch_metric:
                    best_epoch_metric = epoch_new_metric
                    torch.save(self.model.state_dict(),self.exp_name+'.pt')
                    
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        print('Best new metric: {:4f}'.format(best_epoch_metric))

        torch.save(self.model.state_dict(),self.exp_name+'_final.pt')
        
        print('Training completed finally !!!!!')
        
        
    def test_model(self):
                
        self.model.load_state_dict(torch.load(self.exp_name+'.pt'))
        self.model.eval()
        
        mIoU = 0
        running_iou = 0
        total = 0
        mode = 'test'

        image_list = []
        mask_list = []
        pred_list = []
        i=0
        
        with torch.no_grad():
            
            for data in self.dataloaders[mode]:

                images = data['image'].to(self.device)
                mask = data['mask'].to(self.device)
                dist_mask = data['dist_mask']
                
                preds = self.model(images)
                
                out4metric = preds.view(-1)
                mask4metric = mask.view(-1)
                
                prediction = preds[:]
                
                #import pdb;pdb.set_trace()
                sorted_pixels = sorted(prediction.cpu().numpy().reshape(-1))
                
                
                thresh = float(sorted_pixels[int(0.99*len(sorted_pixels))])
                
                print(i)
                i+=1
                prediction[prediction>thresh] = 1
                prediction[prediction<=thresh] = 0
                
                running_iou += get_IoU(prediction.view(-1),mask4metric) * images.size(0)                
                
                base_path = '../Experiments/cbis_ddsm_sup/'
                name = data['name'][0]

                pr = name.replace('.j','_supervised_out_1_percent.j')
                
                preds = preds - preds.min()
                preds = preds/preds.max()
                
                cv2.imwrite(base_path+pr,prediction.cpu().numpy().squeeze()*255)
                #cv2.imwrite(base_path+pr.replace('.j','_probs.j'),preds.cpu().numpy().squeeze()*255)
                
                
        mIoU = running_iou/self.dataset_sizes[mode]
        print('mean IoU:',mIoU)

#                 image_list.append(images.squeeze().cpu().numpy().transpose((1,2,0)))
#                 mask_list.append(mask.transpose((1,2,0)).squeeze())
#                 pred_list.append(preds.squeeze().cpu().numpy())

#         return image_list,mask_list,pred_list
    

In [26]:
u = VGG_unet_bc()

In [27]:
u.train()

Epoch 0/19
----------


train Loss: 0.0872 Dice_Loss: 0.9613 Dist_Loss: 0.2241 IoU: 0.1582 New Metric: 0.3035


valid Loss: 0.0192 Dice_Loss: 0.9590 Dist_Loss: 0.0923 IoU: 0.1734 New Metric: 0.2804
Epoch 1/19
----------


train Loss: 0.0136 Dice_Loss: 0.9511 Dist_Loss: 0.0667 IoU: 0.1994 New Metric: 0.3772


valid Loss: 0.0086 Dice_Loss: 0.9722 Dist_Loss: 0.0429 IoU: 0.0658 New Metric: 0.2121
Epoch 2/19
----------


Process Process-104:
Process Process-103:
Process Process-101:
Process Process-97:
Process Process-99:
Process Process-98:
Process Process-100:
Process Process-102:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/vdslab/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/vdslab/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/vdslab/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/vdslab/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/vdslab/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/vdslab/anac

KeyboardInterrupt: 

  File "/home/vdslab/anaconda3/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/home/vdslab/anaconda3/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
  File "/home/vdslab/anaconda3/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
KeyboardInterrupt
  File "/home/vdslab/anaconda3/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/vdslab/anaconda3/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/home/vdslab/anaconda3/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt


In [18]:
u.test_model()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
plt.imshow(cv2.imread('../Experiments/cbis_ddsm/Mass-Test_P_00056_LEFT_MLO_supervised_out.jpg'))

In [ ]:
plt.imshow(cv2.imread('../Experiments/cbis_ddsm/Mass-Test_P_00056_LEFT_MLO_supervised_new_mask.jpg'))

In [ ]:
u.train()

In [ ]:
%debug

In [ ]:
u.test_model()

In [ ]:
'../Data/masks_orient/Mass-Training_P_00913'

In [ ]:
%debug

In [ ]:
a,b,c = u.test_model()

In [ ]:
len(a)

In [ ]:
plt.imshow(a[1])

In [ ]:
b[0].sum()

In [ ]:
plt.imshow(b[1])

In [ ]:
d = c[1]
d[d>0] = 255
d[d<=0] = 0

In [ ]:
plt.imshow(d)

In [ ]:
get_IoU(d/255,b[1])

In [ ]:
plt.imshow(a[1])

In [ ]:
plt.imshow(b[1])

In [ ]:
plt.imshow(c[1])

In [ ]:
u.train()

In [ ]:
%debug

In [ ]:
_,_,_ = dci.get_cam()

In [ ]:
dci.train()

In [ ]:
!nvidia-smi

In [ ]:
# dci.test_model_acc()

In [ ]:
md,dl = dci.return_model()

In [ ]:
a = iter(dl).next()

m = denorm_img(a['mask'],[0.223, 0.231, 0.243],[0.266, 0.270, 0.274]).squeeze()
bm = denorm_img(a['bmask'],[0.223, 0.231, 0.243],[0.266, 0.270, 0.274]).squeeze()

In [ ]:
device = torch.device("cuda:0")

p = F.sigmoid(md(a['image'].to(device))).detach().cpu().numpy().squeeze()

In [ ]:
m.shape

In [ ]:
p.shape

In [ ]:
bm.shape

In [ ]:
plt.imshow(bm)

In [ ]:
plt.imshow(m)

In [ ]:
p_m = p*bm
p_m[p_m > p_m.mean() + p_m.std()] = 1
p_m[p_m < p_m.mean() + p_m.std()] = 0

In [ ]:
plt.imshow(p_m)

In [ ]:
p_m_t = torch.Tensor(p_m)
print(nn.L1Loss()(torch.Tensor(p_m),torch.zeros(p_m_t.shape)))

In [ ]:
p*bm

In [ ]:
plt.imshow(p)